#LOAD

In [1]:
import numpy as np
import pandas as pd
from sklearn.linear_model import Perceptron
from sklearn.metrics import f1_score
from sklearn.model_selection import GridSearchCV

In [2]:
train = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 2 : PERCEPTRON/ENCODINGS/TRAIN_MEAN.parq')
test = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 2 : PERCEPTRON/ENCODINGS/TEST_MEAN.parq')
val = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 2 : PERCEPTRON/ENCODINGS/VAL_MEAN.parq')

FileNotFoundError: ignored

#PERCEPTRON


In [ ]:
X_train = train.loc[:, ~train.columns.isin(['label', "attacker_ip_enum"])]
y_train = train.loc[:, 'label']
x_test = test.loc[:, ~test.columns.isin(['label', "attacker_ip_enum"])]
y_test = test.loc[:, 'label']
X_val = val.loc[:, ~val.columns.isin(['label', "attacker_ip_enum"])]
y_val = val.loc[:, 'label']
del train
del test

In [ ]:
param_grid = {
    'alpha': [1e-6, 1e-5, 1e-4, 1e-3, 1e-2, 1e-1],
    'eta0': [ 1, 10, 100],
    'max_iter': [10, 100, 500 ,1000],
    'random_state': [1996],
    'class_weight': ['balanced']
}

perceptron = Perceptron()


grid_search = GridSearchCV(perceptron, param_grid, cv=2, scoring='f1', n_jobs=-1, return_train_score = True)
grid_search.fit(X_train, y_train)


best_params = grid_search.best_params_


best_perceptron = Perceptron(**best_params)
best_perceptron.fit(X_train, y_train)


y_test_pred = best_perceptron.predict(x_test)
y_val_pred = best_perceptron.predict(X_val)


f1_test = f1_score(y_test, y_test_pred)
f1_val = f1_score(y_val, y_val_pred)
print("Best Hyperparameters:", best_params)
print("Test Accuracy:", f1_test)
print("Test Accuracy:", f1_val)

Best Hyperparameters: {'alpha': 1e-06, 'class_weight': 'balanced', 'eta0': 1, 'max_iter': 10, 'random_state': 1996}

# muy raro estos numeros debo de estar leakeando target en algun lado para que me vaya asi.

Test Accuracy: 0.6751081594973685

Test Accuracy: 0.6766291295298072

In [ ]:
coefficients = best_perceptron.coef_[0]
column_names = X_train.columns

df = pd.DataFrame({'Column': column_names, 'Coefficient': coefficients})
df

In [ ]:
import matplotlib.pyplot as plt

# Plotting the bar chart
plt.figure(figsize=(10, 6))
plt.bar(df['Column'], df['Coefficient'], color='blue')
plt.xlabel('Columns')
plt.ylabel('Coefficients')
plt.title('Coefficients of Different Columns')
plt.xticks(rotation=45, ha='right')  # Rotate x-axis labels for better readability
plt.tight_layout()

# Display the plot
plt.show()

¿Cuál es el mejor score de validación obtenido? (¿Cómo conviene obtener el dataset para validar?)

Val Accuracy: 0.6751081594973685


Al predecir con este modelo para test, ¿Cúal es el score obtenido? (guardar el csv con predicciones para entregarlo después)

Test Accuracy: 0.6766291295298072


¿Qué features son los más importantes para predecir con el mejor modelo? Graficar.

aparentemente son: attacker as name, watcher as name, attacker country



In [ ]:
FINAL_TEST = pd.read_parquet('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 2 : PERCEPTRON/ENCODINGS/FINAL_MEAN.parq')
len(FINAL_TEST)

In [ ]:
FINAL_TEST

In [ ]:
nan_count_per_column = FINAL_TEST.isna().sum()
nan_count_per_column

In [ ]:
FINAL_PREDICT = best_perceptron.predict(FINAL_TEST.loc[:, ~FINAL_TEST.columns.isin(["attacker_ip_enum"])])

In [ ]:
FINAL_PREDICT = pd.Series(FINAL_PREDICT, name='prediction')
FINAL_PREDICT = pd.concat([FINAL_TEST['attacker_ip_enum'], FINAL_PREDICT], axis=1)

In [ ]:
filtered_result_1 = FINAL_PREDICT[FINAL_PREDICT['prediction'] == 1].drop_duplicates(subset='attacker_ip_enum')
filtered_result_0 = FINAL_PREDICT[FINAL_PREDICT['prediction'] == 0].drop_duplicates(subset='attacker_ip_enum')

del FINAL_PREDICT

# Get IPs not in filtered_result_no_duplicates
filtered_result_0 = filtered_result_0[~filtered_result_0['attacker_ip_enum'].isin(filtered_result_1['attacker_ip_enum'])]



# Concatenate remaining_ips with filtered_result_no_duplicates
final_result = pd.concat([filtered_result_1, filtered_result_0])

del filtered_result_1
del filtered_result_0

final_result

In [ ]:
final_result[final_result["attacker_ip_enum"] == "nan"]

In [ ]:
len(final_result)

In [ ]:
final_result.sort_values(by='attacker_ip_enum', inplace=True)

final_result

In [ ]:
final_result.isna().sum()


In [ ]:
final_result = final_result.rename(columns={'prediction': 'label'})

In [ ]:
final_result = final_result.reset_index(drop=True)
final_result[[]]

In [ ]:
final_result[["attacker_ip_enum", "label"]]

In [ ]:
final_result.reset_index()

In [ ]:
final_result.set_index("attacker_ip_enum").to_csv('/content/drive/MyDrive/Colab Notebooks/TP 2023/TP2/PARTE 2 : PERCEPTRON/FINAL_PREDICT.csv')